In [112]:
import sys
sys.path.append('../../')

from IPython.core.display import display
import numpy as np
import pandas as pd
import networkx as nx
from sortedcontainers import SortedDict
import matplotlib.pyplot as plt
import seaborn as sns
from  matplotlib.ticker import PercentFormatter
import utils.graph_utils as graph_utils
from utils.IO_utils import load_ground_true
from sklearn import metrics

In [113]:
# settings
name = "Dblp"
come_model_type = "GMM"
ks = [2] # [2, 5, 10, 20]

In [114]:
# import graph and true labels

# graph
G = graph_utils.load_matfile(f"../../data/{name}/{name}.mat", undirected=True)

# labels_true
labels_true, _ = load_ground_true(path=f"../../data/{name}", file_name=name)

In [115]:
# import node embeddings and predicted labels
dfs = {}
for k in ks:
    # import node embeddings
    df = pd.read_csv(
        f"./data/{name}_alpha-0.1_beta-0.1_ws-10_neg-5_lr-0.025_icom-219_ind-219_k-{k}_ds-0.0_type-{come_model_type}.txt",
        sep="\t| ",
        header=None
    )
    df = df.rename(columns={0: 'node'})
    df.set_index(['node'], inplace=True)

    # import predicted labels
    labels = pd.read_csv(f"./data/labels_pred_{come_model_type}_{k}.txt", header=None)
    labels = labels.rename(columns={0: 'label'})
    labels.label = labels.astype(int)

    # join labels to embeddings
    df = df.join(labels)

    display(df)
    dfs[k] = df

/Users/anton/miniconda3/envs/MT/lib/python3.6/site-packages/ipykernel_launcher.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


,1,2,3,4,5,6,7,8,9,10,...,120,121,122,123,124,125,126,127,128,label
node,,,,,,,,,,,,,,,,,,,,,
0,0.305006,-0.466880,-0.267545,0.769969,0.285500,0.145671,0.145668,0.589570,0.057389,0.295072,...,0.628949,0.022924,-0.109912,-0.527391,-0.116635,-0.446284,0.117288,-0.009859,-0.156843,1
1,0.274115,-0.487077,-0.116241,0.739823,0.195494,0.060871,0.084048,0.557284,-0.170716,0.435309,...,0.566391,-0.059694,-0.219559,-0.513448,-0.049340,-0.475041,0.052064,0.117735,-0.032222,1
2,0.417771,-0.439148,-0.307630,0.787992,0.405380,0.128854,0.119469,0.588898,0.191887,0.283263,...,0.637316,0.006722,-0.004066,-0.469753,-0.177917,-0.386277,0.056969,0.023117,-0.182263,1
3,0.306671,-0.452776,-0.340008,0.667185,0.238152,0.025103,0.174575,0.546116,0.084083,0.320654,...,0.566142,-0.080180,-0.132030,-0.433514,-0.166257,-0.302751,0.087579,0.046053,-0.229579,1
4,0.218821,-0.380524,-0.374861,0.763850,0.261986,0.098802,0.026748,0.559295,0.217035,0.293235,...,0.611178,0.038992,-0.054769,-0.509383,-0.214571,-0.346377,0.035100,-0.079765,-0.218875,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13179,0.233857,-0.375045,-0.195243,0.585578,0.377580,0.057074,0.045089,0.492758,0.113148,0.298501,...,0.568135,-0.154959,-0.091873,-0.352352,-0.144252,-0.037096,0.055319,0.095775,-0.173232,1
13180,-0.128917,-0.556151,-0.205546,-0.030483,0.330267,0.210279,0.195406,0.034497,0.400683,0.501986,...,0.165083,-0.314047,0.298250,-0.008569,0.241477,-0.022174,-0.508296,0.082054,0.339728,0
13181,-0.181398,-0.531019,-0.020660,0.022836,0.288119,0.118601,0.141081,0.019796,0.511212,0.564311,...,0.170675,-0.129990,0.343729,-0.115812,0.100923,-0.177653,-0.494129,0.069173,0.356277,0


# Community Detection

In [116]:
# NMI

for k in dfs:
    print("K =", k)
    df = dfs[k]
    labels_pred = df.label.to_numpy()

    nmi = metrics.normalized_mutual_info_score(labels_true, labels_pred)
    print("NMI: ", nmi)

K = 2
NMI:  0.42676536769046264


In [117]:
# Conductance

for k in dfs:
    print("K =", k)
    df = dfs[k]
    conductance_min = 1
    for i in range(k):
        nodes = df[df.label == i].index.to_numpy()
        conductance_i = nx.conductance(G, nodes)
        print(f"  i{i} conductance: {conductance_i}")
        conductance_min = min(conductance_min, conductance_i)
    print(f"=>conductance: {conductance_min}")

K = 2
  i0 conductance: 0.03589380074751901
  i1 conductance: 0.03589380074751901
=>conductance: 0.03589380074751901


# Node Classification

In [118]:
# train LibSVM classifier on 90% of data and test on 10%.

#TODO

In [119]:
def emb_from_row(row):
    """get node embedding np array from df row"""
    return row.drop('label').to_numpy().flatten()
